# Analysis
## Strategy

1. Identify genes with significant changes in expression.
2. Identify zones with significant changes in accessibility.
3. Detect hotspots in accessibility changes over gene regulatory areas of differentially expressed genes.
4. Detect enriched TF motifs in zones presenting accessibility changes.
5. Detect enriched TF motifs in hotspots.
6. Perform GO Analysis to put genes in context.

## AME Analysis of Motif Enrichment
### Defining windows for ROIs
**High Access / Upregulated**
- Window 0: -2000 to 0  
- Window 1: -4250 to -3250  
- Window 2: -8000 to -4500
- Window 3: -9750 to -8500  

In [1]:
#!awk 'BEGIN{OFS="\t";offset5=-2000;offset3=0} NR==FNR{chr[$1]=$2;next} {if($1 in chr){lim=chr[$1];if($6=="+"){left=$2+offset5;right=$2+offset3}else{left=$3-offset3;right=$3-offset5}; if(left<0){left=0}else if(right>lim){right=lim}else{$2=left;$3=right}; print}else{print "error "$1" not in genome"}}' output/GRCh38.genome output/upregby_hiprom.bed | bedtools intersect -a - -b output/atac_hi.bed | sort -k1,1 -k2,2n | bedtools getfasta -name -s -fi external/GRCh38.fa -bed - | awk '/^>/{$0=$0"_"(++i)}1' > output/upregby_hiprom_win0.fasta

The pipeline above has been converted into a bash script: `scripts/tssreg.sh` to make it clearer and easier to experiment with

In [11]:
!scripts/tssreg.sh -i output/upregby_hiprom.bed -p output/atac_hi.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-2000" -r 0 > output/upregby_hiprom_win0.fasta

In [ ]:
!scripts/tssreg.sh -i output/upregby_hiprom.bed -p output/atac_hi.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-4250" -r "-3250" > output/upregby_hiprom_win1.fasta

In [ ]:
!scripts/tssreg.sh -i output/upregby_hiprom.bed -p output/atac_hi.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-8000" -r "-4500" > output/upregby_hiprom_win2.fasta

In [ ]:
!scripts/tssreg.sh -i output/upregby_hiprom.bed -p output/atac_hi.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-9750" -r "-8500" > output/upregby_hiprom_win3.fasta

**High Access / Downregulated**
- Window 0: -2000 to 0  
- Window 1: -9500 to -4500  

In [ ]:
!scripts/tssreg.sh -i output/dwregby_hiprom.bed -p output/atac_hi.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-2000" -r "0" > output/dwregby_hiprom_win0.fasta

In [ ]:
!scripts/tssreg.sh -i output/dwregby_hiprom.bed -p output/atac_hi.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-9500" -r "-4500" > output/dwregby_hiprom_win1.fasta

**Low Access / Upregulated**
- Window 0: -3000 to 0  
- Window 1: -6000 to -4500  
- Window 2: -9750 to -8500

In [ ]:
!scripts/tssreg.sh -i output/upregby_loprom.bed -p output/atac_lo.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-3000" -r "0" > output/upregby_loprom_win0.fasta

In [ ]:
!scripts/tssreg.sh -i output/upregby_loprom.bed -p output/atac_lo.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-6000" -r "-4500" > output/upregby_loprom_win1.fasta

In [ ]:
!scripts/tssreg.sh -i output/upregby_loprom.bed -p output/atac_lo.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-9750" -r "-8500" > output/upregby_loprom_win2.fasta

**Low Access / Downregulated**
- Window 0: -3000 to 0   
- Window 1: -9750 to -8250

In [ ]:
!scripts/tssreg.sh -i output/dwregby_loprom.bed -p output/atac_lo.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-3000" -r "0" > output/dwregby_loprom_win0.fasta

In [ ]:
!scripts/tssreg.sh -i output/dwregby_loprom.bed -p output/atac_lo.bed -g output/GRCh38.genome -f external/GRCh38.fa -l "-9750" -r "-8250" > output/dwregby_loprom_win1.fasta

All High Access Regions  
All Low Access Regions  
All Regions

In [12]:
!bedtools getfasta -name -fi external/GRCh38.fa -bed output/atac_hi.bed > output/atac_hi.fasta
!awk '{/>/&&++a||b+=length()}END{print b/a,a}' output/atac_hi.fasta
!bedtools getfasta -name -fi external/GRCh38.fa -bed output/atac_lo.bed > output/atac_lo.fasta
!awk '{/>/&&++a||b+=length()}END{print b/a,a}' output/atac_lo.fasta
!bedtools getfasta -name -fi external/GRCh38.fa -bed output/atac_diffx.bed > output/atac_diffx.fasta
!awk '{/>/&&++a||b+=length()}END{print b/a,a}' output/atac_diffx.fasta
!bedtools getfasta -name -fi external/GRCh38.fa -bed output/atacseq_fpeaks.bed6 > output/atac_fpeaks.fasta
!awk '{/>/&&++a||b+=length()}END{print b/a,a}' output/atac_fpeaks.fasta

741.084 16971
929.923 11618
817.825 28589
685.638 173264


### Generating backgrounds for AME
Creating a random background

In [13]:
!bedtools random -l 100 -n 100000 -seed 42 -g output/GRCh38.genome > output/atac_bgd.bed 2> /dev/null
!bedtools getfasta -name -fi external/GRCh38.fa -bed output/atac_bgd.bed > output/atac_bgd.fasta
!awk '{/>/&&++a||b+=length()}END{print b/a,a}' output/atac_bgd.fasta

100 100000


### AME: General and Differential Landscape - Enriched Motifs

In [14]:
!ame --o output/AME/total_VS_bgd --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_bgd.fasta output/atac_fpeaks.fasta external/JASPAR2018_HUMAN_meme.txt

Using average odds scoring.
Added external/JASPAR2018_HUMAN_meme.txt to motif_sources which now has 1 file names.
Motif file name is external/JASPAR2018_HUMAN_meme.txt.
Writing results to output directory 'output/AME/total_VS_bgd'.
E-value threshold for reporting results: 0.05
Checking alphabets in 1 motif files.
Loading motifs from file 'external/JASPAR2018_HUMAN_meme.txt'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (173264).
MOTIF: 1 SEQ: 273264/273264
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 639 SEQ: 273264/273264


In [15]:
!ame --o output/AME/diffx_VS_total --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_fpeaks.fasta output/atac_diffx.fasta external/JASPAR2018_HUMAN_meme.txt

Using average odds scoring.
Added external/JASPAR2018_HUMAN_meme.txt to motif_sources which now has 1 file names.
Motif file name is external/JASPAR2018_HUMAN_meme.txt.
Writing results to output directory 'output/AME/diffx_VS_total'.
E-value threshold for reporting results: 0.05
Checking alphabets in 1 motif files.
Loading motifs from file 'external/JASPAR2018_HUMAN_meme.txt'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (28589).
MOTIF: 1 SEQ: 201853/201853
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 639 SEQ: 201853/201853


### AME: High/Low Accessibility Motifs Enrichment

Note: I have selected Human Only Motifs from JASPAR  
http://jaspar.genereg.net  
File is provided in external/JASPAR2018_HUMAN_meme.txt

In [16]:
!ame --o output/AME/hi_VS_diffx --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_diffx.fasta output/atac_hi.fasta external/JASPAR2018_HUMAN_meme.txt

Using average odds scoring.
Added external/JASPAR2018_HUMAN_meme.txt to motif_sources which now has 1 file names.
Motif file name is external/JASPAR2018_HUMAN_meme.txt.
Writing results to output directory 'output/AME/hi_VS_diffx'.
E-value threshold for reporting results: 0.05
Checking alphabets in 1 motif files.
Loading motifs from file 'external/JASPAR2018_HUMAN_meme.txt'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (16971).
MOTIF: 1 SEQ: 45560/45560
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 639 SEQ: 45560/45560


In [17]:
!ame --o output/AME/lo_VS_diffx --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_diffx.fasta output/atac_lo.fasta external/JASPAR2018_HUMAN_meme.txt

Using average odds scoring.
Added external/JASPAR2018_HUMAN_meme.txt to motif_sources which now has 1 file names.
Motif file name is external/JASPAR2018_HUMAN_meme.txt.
Writing results to output directory 'output/AME/lo_VS_diffx'.
E-value threshold for reporting results: 0.05
Checking alphabets in 1 motif files.
Loading motifs from file 'external/JASPAR2018_HUMAN_meme.txt'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (11618).
MOTIF: 1 SEQ: 40207/40207
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 639 SEQ: 40207/40207


### AME Regional Enrichment with Up/Down Regulated Genes

**High Access / Upregulated** Only Window 0 has hits
- Window 0: -2000 to 0  
- Window 1: -4250 to -3250  
- Window 2: -8000 to -4500
- Window 3: -9750 to -8500  

In [18]:
!ame --o output/AME/uphiwin0_VS_hi --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_hi.fasta output/upregby_hiprom_win0.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/uphiwin1_VS_hi --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_hi.fasta output/upregby_hiprom_win1.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/uphiwin2_VS_hi --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_hi.fasta output/upregby_hiprom_win2.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/uphiwin3_VS_hi --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_hi.fasta output/upregby_hiprom_win3.fasta external/JASPAR2018_HUMAN_meme.txt

Using average odds scoring.
Added external/JASPAR2018_HUMAN_meme.txt to motif_sources which now has 1 file names.
Motif file name is external/JASPAR2018_HUMAN_meme.txt.
Writing results to output directory 'output/AME/uphiwin0_VS_hi'.
E-value threshold for reporting results: 0.05
Checking alphabets in 1 motif files.
Loading motifs from file 'external/JASPAR2018_HUMAN_meme.txt'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (85).
MOTIF: 1 SEQ: 17056/17056
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 639 SEQ: 17056/17056


**High Access / Downregulated** (No hits)
- Window 0: -2000 to 0  
- Window 1: -9500 to -4500 

In [19]:
#!ame --o output/AME/dwhiwin0_VS_hi --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_hi.fasta output/dwregby_hiprom_win0.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/dwhiwin1_VS_hi --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_hi.fasta output/dwregby_hiprom_win1.fasta external/JASPAR2018_HUMAN_meme.txt

**Low Access / Upregulated** No Hits
- Window 0: -3000 to 0  
- Window 1: -6000 to -4500  
- Window 2: -9750 to -8500

In [20]:
#!ame --o output/AME/uplowin0_VS_lo --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_lo.fasta output/upregby_loprom_win0.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/uplowin1_VS_lo --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_lo.fasta output/upregby_loprom_win1.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/uplowin2_VS_lo --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_lo.fasta output/upregby_loprom_win2.fasta external/JASPAR2018_HUMAN_meme.txt

**Low Access / Downregulated** No Hits
- Window 0: -3000 to 0   
- Window 1: -9750 to -8250

In [21]:
#!ame --o output/AME/dwlowin0_VS_diffx --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_diffx.fasta output/dwregby_loprom_win0.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/dwlowin0_VS_lo --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_lo.fasta output/dwregby_loprom_win0.fasta external/JASPAR2018_HUMAN_meme.txt
#!ame --o output/AME/dwlowin1_VS_lo --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 0.05 --control output/atac_lo.fasta output/dwregby_loprom_win1.fasta external/JASPAR2018_HUMAN_meme.txt

Using average odds scoring.
Added external/JASPAR2018_HUMAN_meme.txt to motif_sources which now has 1 file names.
Motif file name is external/JASPAR2018_HUMAN_meme.txt.
Writing results to output directory 'output/AME/dwlowin0_VS_diffx'.
E-value threshold for reporting results: 0.05
Checking alphabets in 1 motif files.
Loading motifs from file 'external/JASPAR2018_HUMAN_meme.txt'
Loading primary sequences.
Loading control sequences.
Not in partition maximization mode. Fixing partition at the number of primary sequences (3).
MOTIF: 1 SEQ: 28592/28592
Sorting sequences by sequence PWM score to get PWM ranks; breaking ties to put negatives first.
Leaving sequences sorted by PWM score.
Optimizing over sequence PWM score threshold.
MOTIF: 639 SEQ: 28592/28592


Clean Up Results

In [22]:
!awk 'BEGIN{OFS="\t"} $1 !~ "#" {split($4, a, "::"); for (k in a) {print $1,$3,a[k],$7,$8};delete a}' output/AME/hi_VS_diffx/ame.tsv | sed 's/'\(.*\)'//g' > output/hi_TFMs.tsv
!awk 'BEGIN{OFS="\t"} $1 !~ "#" {split($4, a, "::"); for (k in a) {print $1,$3,a[k],$7,$8};delete a}' output/AME/lo_VS_diffx/ame.tsv | sed 's/'\(.*\)'//g' > output/lo_TFMs.tsv
!awk 'BEGIN{OFS="\t"} $1 !~ "#" {split($4, a, "::"); for (k in a) {print $1,$3,a[k],$7,$8};delete a}' output/AME/uphiwin0_VS_hi/ame.tsv | sed 's/'\(.*\)'//g' > output/uphiprox_TFMs.tsv